## Preparation and QC of UKB participants

In [247]:
import os
import sys
import pandas as pd
import subprocess
import numpy as np
rootdir = '/home/preclineu/ramcir/Desktop/Diffusion/Subs'
#os.path.exists(rootdir)

In [297]:
def prep_sub(rootdir, id_sub):
    subdir = os.path.join(rootdir, id_sub, 'dMRI', 'dMRI')
    mask_dir = os.path.join(rootdir, id_sub, 'dMRI', 'dMRI', 'nodif_mask.nii.gz')
    acqparam_file = os.path.join(rootdir, id_sub, 'dMRI', 'dMRI', 'acqparam.txt')
    idx_file = os.path.join(rootdir, id_sub, 'dMRI', 'dMRI', 'index.txt')
    
    #Create symbolic link for data_ud with the name data
    #This is necessary for running 'quad' since all other eddy outputs are called data.eddy.-
    data_ud = os.path.join(subdir, 'data_ud.nii.gz')
    data = os.path.join(subdir, 'data.nii.gz')
    
    if os.path.islink(data) == False:
        os.symlink(data_ud, data)
    else:
        pass

    #Write command for creating the b0 mask and run the bash process
    if os.path.exists(mask_dir) == False:
        fslroi_cmd = 'fslroi data.nii.gz b0_data.nii.gz 0 1'
        fslbet_cmd = 'bet b0_data.nii.gz nodif -f 0.1 -g 0 -m -n'
        prep_cmd = "cd " + subdir + " && " + fslroi_cmd + " && " + fslbet_cmd 
        subprocess.run(prep_cmd, shell = True)
    else:
        pass

    #Create aquision parameters file and index file
    #The acquisition parameters file is based on dataset acq documentation
    #i.e. AP dir = 0 1 0, PA dir = 0 -1 0, Echo spacing = 0.67 ms, EPI factor = 104 (for UKB)
    if os.path.exists(acqparam_file) == False:
        with open(os.path.join(subdir,'acqparams.txt'), 'w') as acqparams:
            acqparams.write('0 1 0 0.069\n0 -1 0 0.069')
    else:
        pass

    #The index file indicated the direction in which each volume was aquired (based on aqcparams.txt)
    #count number of volumes present in data
    if os.path.exists(idx_file) == False:
        with open(os.path.join(subdir,'bvals'), 'r') as bvals:
            data = bvals.read()
            vols = data.split()
            no_vols = len(vols)
        with open(os.path.join(subdir,'index.txt'), 'w') as index:
            for vol in range(0,no_vols):    
                index.write('1 ')
    else:
        pass

In [298]:
def quad_sub(rootdir, id_sub, quad_folders):  
    subdir = os.path.join(rootdir, id_sub, 'dMRI', 'dMRI')
    qc_dir = os.path.join(subdir,'data.qc') 
    
    #check to see if quad has already been run
    if os.path.exists(qc_dir) == False:        
        #compose and run quad command
        quad_cmd = 'eddy_quad data  -idx index.txt -par acqparams.txt -m nodif_mask -b bvals -g bvecs'
        prep_cmd = "cd " + subdir + " && " + quad_cmd
        process = subprocess.run(prep_cmd, shell = True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)        
    else:
        print('QUAD already exists')     

### Run QUAD on multiple participants

In [ ]:
# Read the sorted participants dataframe, extract the ready column 
# then covert sub IDs into strings and remove nans
ready_subs = pd.read_csv(os.path.join(rootdir,'sorted_subs.csv'), usecols = ['ready'])
ready_subs = ready_subs['ready'].apply(lambda sub: sub if pd.isnull(sub) else str(int(sub))).tolist()
ready_subs = [sub for sub in ready_subs if str(sub) != 'nan']

# Apply prep and quad functions to all particpants in the ready list
for id_sub in ready_subs:
    print('Participant ',id_sub, 'is being processed')
    print('Preparing files')
    prep_sub(rootdir, id_sub)
    print('Running QUAD')
    quad_sub(rootdir, id_sub, quad_folders) 
        
print('Finished processing QUAD.')  

In [300]:
#create file to store the paths to the quad folder of each participant
with open(os.path.join(rootdir,'quad_folders.txt'), 'w') as quad_folders:
    for id_sub in ready_subs:
        subdir = os.path.join(rootdir, id_sub, 'dMRI', 'dMRI')
        qc_dir = os.path.join(subdir,'data.qc')
        if os.path.exists(qc_dir) == True:       
             quad_folders.writelines(qc_dir + '\n')

In [323]:
def squad_sub(rootdir):    
    if os.path.exists(qc_dir) == True:
        squad_cmd = 'eddy_squad'
        quad_folders = os.path.join(rootdir,'quad_folders.txt')
        prep_cmd = "cd " + rootdir + " && " + squad_cmd + ' ' + quad_folders
        print(prep_cmd)
        process = subprocess.run(prep_cmd, shell = True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         out, err = process.communicate()
#         print(out)
#         print(err)
    else:
        print('QUAD folder not there')

In [324]:
squad_sub(rootdir)

cd /home/preclineu/ramcir/Desktop/Diffusion/Subs && eddy_squad /home/preclineu/ramcir/Desktop/Diffusion/Subs/quad_folders.txt
